### TRAIN

In [1]:
import mmfewshot
import mmdet

In [2]:
shots = [5]
xml_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/data/VOCdevkit/VOC2012/Annotations"
image_xml_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/data/VOCdevkit/VOC2012/JPEGImages"
few_shot_ann_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/data/few_shot_ann/voc"

In [3]:
from PIL import Image

def get_xml_content(cat, img_path, x1, y1, x2, y2):
  img_name = img_path.split("/")[-1]
  img = Image.open(img_path)
  w, h = img.size
  content = f'''
<annotation>
<filename>{img_name}</filename>
<folder>VOC2012</folder>
<object>
  <name>{cat}</name>
  <actions>
    <jumping>0</jumping>
    <other>1</other>
    <phoning>0</phoning>
    <playinginstrument>0</playinginstrument>
    <reading>0</reading>
    <ridingbike>0</ridingbike>
    <ridinghorse>0</ridinghorse>
    <running>0</running>
    <takingphoto>0</takingphoto>
    <usingcomputer>0</usingcomputer>
    <walking>0</walking>
  </actions>
  <bndbox>
    <xmax>{x2}</xmax>
    <xmin>{x1}</xmin>
    <ymax>{y2}</ymax>
    <ymin>{y1}</ymin>
  </bndbox>
  <difficult>0</difficult>
  <pose>Unspecified</pose>
  <point>
    <x>308</x>
    <y>189</y>
  </point>
</object>
<segmented>0</segmented>
<size>
  <depth>3</depth>
  <height>{h}</height>
  <width>{w}</width>
</size>
<source>
  <annotation>PASCAL VOC2012</annotation>
  <database>The VOC2012 Database</database>
  <image>flickr</image>
</source>
</annotation>
  '''.strip()
  return content

In [4]:
import mmdet
import os
import shutil
import cv2
import numpy as np

def get_bb_contour(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    ## (2) Morph-op to remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

    ## (3) Find the max-area contour
    cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    cnt = sorted(cnts, key=cv2.contourArea)[-1]

    ## (4) Crop and save it
    x,y,w,h = cv2.boundingRect(cnt)
    return x, y, x+w, y+h

def get_bb_full_image(img_path):
    img = cv2.imread(img_path)
    return 1, 1, img.shape[1], img.shape[0]

def get_max_object(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    ## (2) Morph-op to remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

    ## (3) Find the max-area contour
    cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    x1 = []
    y1 = []
    x2 = []
    y2 = []
    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
        x1.append(x)
        y1.append(y)
        x2.append(x+w)
        y2.append(y+h)
    x1 = min(x1)
    y1 = min(y1)
    x2 = max(x2)
    y2 = max(y2)
    return x1, y1, x2, y2


def create_annotations_and_label_images(cat, query_image_path,
                                        label_image_path,
                                        ):
    color = (0, 0, 255)
    thickness = 2
    if not os.path.exists(os.path.join(label_image_path, "img_with_bbox")):
        os.makedirs(os.path.join(label_image_path, "img_with_bbox"))

    for idx, img_name in enumerate(os.listdir(query_image_path)):
        img_path = os.path.join(query_image_path, img_name)
        # x1, y1, x2, y2 = get_bb_contour(img_path)
        x1, y1, x2, y2 = get_max_object(img_path)
        
        img = cv2.imread(img_path)
        img = cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
        new_name = img_name.split(".")[0] + "_bbox." + img_name.split(".")[1]
        new_img_path = os.path.join(label_image_path, "img_with_bbox", new_name)
        cv2.imwrite(new_img_path, img)
        xml_content = get_xml_content(cat, img_path, x1, y1, x2, y2)
        xml_name = img_name.split(".")[0]
        if os.path.exists(os.path.join(xml_path, xml_name + ".xml")):
            os.remove(os.path.join(xml_path, xml_name + ".xml"))
        f = open(os.path.join(xml_path, xml_name + ".txt"), 'w')
        f.write(xml_content)
        f.close()
        os.rename(os.path.join(xml_path, xml_name + ".txt"), os.path.join(xml_path, xml_name + ".xml"))
        
        shutil.copy(os.path.join(query_image_path, img_name), os.path.join(image_xml_path, img_name))

In [5]:
def set_category(cat):
    with open('/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/mmfewshot/detection/datasets/voc.py', 'r') as file:
        file_content = file.readlines()

    file_content[23] = f"\t\t\t\t\t'pottedplant', 'sheep', 'train', 'tvmonitor', '{cat}'),\n"
    file_content[33] = f"\tNOVEL_CLASSES_SPLIT1=('{cat}',),\n"

    with open('/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/mmfewshot/detection/datasets/voc.py', 'w') as file:
        file.writelines(file_content)
    print(file_content[33])

In [6]:
def add_image_to_txt_path(cat, query_image_path):
  for shot in shots:
      if (len(os.listdir(query_image_path))) < shot:
          print(f"[INFO] not enough image for {shot} shot")
          break
      print(shot)
      f = open(os.path.join(few_shot_ann_path, f"benchmark_{str(shot)}shot/box_{str(shot)}shot_{cat}_train.txt"), "w")
      for img_name in os.listdir(query_image_path)[:shot]:
        f.write(f"VOC2012/JPEGImages/{img_name}\n")
      f.close()

In [7]:
import subprocess

data_dict = [
    # {
    #     'cat': '3326',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3326/focus_3326",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3326/3326",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3326/",
    # },
    # {
    #     'cat': '3191',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3191/focus_3191",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3191/3191",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3191/",
    # },
    # {
    #     'cat': 'door_focus_5_no_background',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/img/focus_door/no_background",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/img/door",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/img/",
    # },
    # {
    #     'cat': 'kettlebell',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/image/KETTLEBELL/focus_Kettlebell",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/image/KETTLEBELL/Kettlebell",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/image/KETTLEBELL/",
    # },
    # {
    #     'cat': '3190',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3190/focus_3190",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3190/3190",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3190/",
    # },
    # {
    #     'cat': '2930',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2930/focus_2930",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2930/2930",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2930/",
    # },
    # {
    #     'cat': '2864',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2864/focus_2864",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2864/2864",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2864/",
    # },
    # {
    #     'cat': '3264',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3264/focus_3264",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3264/3264",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3264/",
    # },
    # {
    #     'cat': 'papasan',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/image/GAR-514/focus_GAR514",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/image/GAR-514/GAR514",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/image/GAR-514",
    # },
    # {
    #     'cat': 'tv_stand',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2052/focus_2052",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2052/2052",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2052",
    # },
    # {
    #     'cat': 'tv_stand_2053',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2053/focus_2053",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2053/2053",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2053",
    # },
    # {
    #     'cat': '3884',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3884/focus_3884",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3884/3884",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3884",
    # },
    # {
    #     'cat': '3883',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3883/focus_3883",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3883/3883",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3883",
    # },
    # {
    #     'cat': '3848',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848/focus_3848",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848/3848",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848",
    # },
    # {
    #     'cat': '3963',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3963/focus_3963/focus_1",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3963/3963",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3963",
    # },
    # {
    #     'cat': '3865',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3865/focus_1",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3865/3865",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3865",
    # },
    # {
    #     'cat': '3846',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3846/focus_1",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3846/3846",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3846",
    # },
    # {
    #     'cat': '3771',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3771/focus_3771",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3771/3771",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3771",
    # },
    # {
    #     'cat': '2030',
    #     'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2030/focus_2030",
    #     'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2030/2030",
    #     'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/2030",
    # },
    {
        'cat': '3848',
        'query_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848/focus_3848",
        'database_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848/3848",
        'label_image_path': "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848",
    },
]

work_dir = "work_dirs/additional_focus"

In [8]:
for data in data_dict:
  print("Processing data ", data['cat'], "...")
  create_annotations_and_label_images(data['cat'], data['query_image_path'],
                                      data['label_image_path'])
  set_category(data['cat'])
  add_image_to_txt_path(data['cat'], data['query_image_path'])
  command = f"CUDA_VISIBLE_DEVICES=0 python tools/detection/train.py configs/detection/attention_rpn/voc/split1/attention-rpn_r50_c4_voc-split1_5shot-fine-tuning.py --work-dir {os.path.join(work_dir, data['cat'])} --no-validate"
  print(command)
  result = subprocess.run(command, shell=True) 

Processing data  3848 ...
	NOVEL_CLASSES_SPLIT1=('3848',),

5
CUDA_VISIBLE_DEVICES=0 python tools/detection/train.py configs/detection/attention_rpn/voc/split1/attention-rpn_r50_c4_voc-split1_5shot-fine-tuning.py --work-dir work_dirs/additional_focus/3848 --no-validate


2023-05-23 14:17:16,557 - mmfewshot - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.10 (default, Jun  4 2021, 15:09:15) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA GeForce RTX 3060
CUDA_HOME: /usr/local/cuda-11.8
NVCC: Cuda compilation tools, release 11.8, V11.8.89
GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 2.0.0+cu118
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;a

Training time: 7m 42.7s

In [8]:
command = f"CUDA_VISIBLE_DEVICES=0 python tools/detection/train.py /home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/tv_stand_attention-rpn_5shot-fine-tuning/tv_stand_attention-rpn_5shot-fine-tuning.py --no-validate"
result = subprocess.run(command, shell=True) 

Traceback (most recent call last):
  File "tools/detection/train.py", line 237, in <module>
    main()
  File "tools/detection/train.py", line 105, in main
    cfg = Config.fromfile(args.config)
  File "/home/tanluuuuuuu/miniconda3/envs/mmfewshot/lib/python3.8/site-packages/mmcv/utils/config.py", line 340, in fromfile
    cfg_dict, cfg_text = Config._file2dict(filename,
  File "/home/tanluuuuuuu/miniconda3/envs/mmfewshot/lib/python3.8/site-packages/mmcv/utils/config.py", line 183, in _file2dict
    check_file_exist(filename)
  File "/home/tanluuuuuuu/miniconda3/envs/mmfewshot/lib/python3.8/site-packages/mmcv/utils/path.py", line 23, in check_file_exist
    raise FileNotFoundError(msg_tmpl.format(filename))
FileNotFoundError: file "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/tv_stand_attention-rpn_5shot-fine-tuning/tv_stand_attention-rpn_5shot-fine-tuning.py" does not exist


### INFERENCE

In [33]:
folder_workdir = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/pipeline_v3/test_default_config_max_object"
print(os.listdir(folder_workdir))

['door', '2930', 'kettlebell', '2864', '3190', '3191', '3264', '3326']


In [ ]:
from mmfewshot.detection.apis import init_detector, inference_detector
import mmcv
import os
from tqdm import tqdm
import time
import cv2
import torch
import torchvision
import numpy as np

THR = 0.5
THR_NMS = 0.2

folder_workdir = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/meta-rcnn"
save_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_v4/meta-rcnn"
for data in data_dict:
  # Specify the path to model config and checkpoint file
  config_file = os.path.join(folder_workdir, data['cat'], "meta-rcnn_r50_c4_8xb4_voc-split1_5shot-fine-tuning.py")
  checkpoint_file = os.path.join(folder_workdir, data['cat'], "iter_800.pth")

  # build the model from a config file and a checkpoint file
  model = init_detector(config_file, checkpoint_file, device='cuda:0')

  img_path = data['database_image_path']
  cat = data['cat']
  # breakpoint()
  num_iter = int(checkpoint_file.split("/")[-1].split(".")[0].split("_")[1])
  folder_name = f"{cat}_{num_iter}iter_thr{str(int(THR*100))}_nms{str(int(THR_NMS*100))}"
  print(folder_name)
  include_path = os.path.join(save_path, folder_name, "include")
  if not os.path.exists(include_path):
    print(f"Make dir {include_path}")
    os.makedirs(include_path)

  exclude_path = os.path.join(save_path, folder_name, "exclude")
  if not os.path.exists(exclude_path):
    print(f"Make dir {exclude_path}")
    os.makedirs(exclude_path)

  sum_time = 0

  def nms(boxes):
    boxbox = torch.from_numpy(boxes[:, 0:4])
    score = torch.from_numpy(boxes[:, 4])
    result = torchvision.ops.nms(boxbox, score, THR_NMS)
    return result.numpy()

  for im_name in tqdm(os.listdir(img_path)[:500]):
    img = os.path.join(img_path, im_name)
    st = time.time()
    try:
      result = inference_detector(model, img)
    except:
      print(im_name)
      continue
    sum_time += (time.time() - st)
    img = cv2.imread(img)
    boxes = np.array([box for box in result[-1] if box[4] >= THR])
    if len(boxes) > 0:
      box_idxs = nms(boxes)
      for box in boxes[box_idxs]:
        score = box[4]
        if score >= THR:
          box = box.astype(int)
          color = (0, 0, 255)
          thickness = 2
          img = cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, thickness)
          img = cv2.putText(img, str(score), (box[0], box[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1, cv2.LINE_AA)
      cv2.imwrite(os.path.join(include_path, im_name), img)
    else:
      cv2.imwrite(os.path.join(exclude_path, im_name), img)
  print(f"Infer on {len(os.listdir(img_path))} got {sum_time}s")


GPU RAM inference: 2074MiB

Time: 17m 22.1s

### MISCELLANEOUS

In [ ]:
#!/usr/bin/python3
# Created by Silencer @ Stackoverflow 
# 2018.01.23 14:41:42 CST
# 2018.01.23 18:17:42 CST
import cv2
import numpy as np

## (1) Convert to gray, and threshold
img = cv2.imread("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/photo_2023-04-11_08-17-05.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

## (2) Morph-op to remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

## (3) Find the max-area contour
cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
cnt = sorted(cnts, key=cv2.contourArea)[-1]

## (4) Crop and save it
x,y,w,h = cv2.boundingRect(cnt)
# dst = img[y:y+h, x:x+w]
color = (255, 0, 0)
thickness = 1
img = cv2.rectangle(img, (y, x), (y+h, x+w), color, thickness)
cv2.imshow("img.jpg", img)
# cv2.imwrite("001.png", dst)

In [ ]:
import mmdet

In [13]:
import os
import shutil

folder1 = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_v0/door_new_bbox_4500iter_thr20/include"
folder2 = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/fine_tune_v0/door_new_bbox_4500iter_th10_nms40/include"

list_1 = os.listdir(folder1)
list_2 = os.listdir(folder2)

diff_folder = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_v2/diff_20_10"

list_diff = list(set(list_2) - set(list_1))
for im in list_diff:
    source = os.path.join(folder2, im)
    dest = os.path.join(diff_folder, im)
    shutil.copy(source, dest)

In [ ]:
a, b = map(int, input().split())
def recur(a, b):
    if b == 0:
        return a
    return recur(b, b % a)
c = recur(a, b)
print(c)

In [47]:
from mmfewshot.detection.apis import init_detector, inference_detector
import cv2

config_file = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/pipeline_v3/test_default_config_max_object/door/fsce_r101_fpn_contrastive-loss_voc-split1_5shot-fine-tuning.py"
checkpoint_file = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/pipeline_v3/test_default_config_max_object/door/iter_4500.pth"

model = init_detector(config_file, checkpoint_file, device='cuda:0')
# print(model)
img_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/door_attention_rpn/exclude/B096LGD5BR.jpg"
result = inference_detector(model, img_path)
print(result)
# img = cv2.imread(img_path)
# print(img.shape)
# for box in result[-1]:
#   if box[-1] >= 0.5:
#     box = list(map(int, box))
#     print(box)
#     cropped = img[box[1]:box[3], box[0]:box[2]]
#     cv2.imwrite("001.png", cropped)

load checkpoint from local path: /home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/pipeline_v3/test_default_config_max_object/door/iter_4500.pth
[array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([[  0.       ,  16.0888   , 237.56364  , 249.24487  ,   0.9340162]],
      dtype=float32)]


In [4]:
import cv2

img1 = cv2.imread("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/attention_rpn/door_40thr_attention_rpn/exclude/B0BKTFJ5SY.jpg")
img2 = cv2.imread("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/img/door/B0BKTFJ5SY.jpg")
print(img1.shape)
print(img2.shape)
print(img1[0][0])
print(img2[0][0])

(250, 250, 3)
(250, 250, 3)
[155 182 203]
[143 166 181]
